In [1]:
import torchaudio
import torch
from dataloader import ESC50
from pathlib import Path
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset
from torchvision import transforms
import torchvision
from matplotlib import pyplot as plt
import time

/home/ifire/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/ifire/.local/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
BATCH_SIZE = 64

In [ ]:
test_esc = ESC50("/home/ifire/learnpytorh/audio/ESC-50/audio")
tensor, label = list(test_esc)[0]

In [ ]:
tensor.shape, label

In [ ]:
sample_dataloader = torch.utils.data.DataLoader(test_esc, batch_size=64, shuffle=True)

In [ ]:
Path.cwd() 

In [ ]:
PATH_TO_ESC50 = Path.cwd() / 'ESC-50/audio'
PATH_TO_ESC50

In [ ]:
train_es50 = ESC50(PATH_TO_ESC50/ 'train')
valid_es50 = ESC50(PATH_TO_ESC50/ 'valid')
test_es50 = ESC50(PATH_TO_ESC50/ 'test')


In [ ]:
train_loader = torch.utils.data.DataLoader(train_es50, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_es50, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_es50, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
## core model 
class AudioNet(nn.Module):
    def __init__(self) -> None:
        super(AudioNet, self).__init__()
        self.conv1 = nn.Conv1d(1,128,80,4)
        self.bn1 = nn.BatchNorm1d(128)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(128,128,80,3)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(128,256,80,3)
        self.bn3 = nn.BatchNorm1d(256)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(256,512,80,3)
        self.bn4 = nn.BatchNorm1d(512)
        self.pool4 = nn.MaxPool1d(4)
        self.avgpool = nn.AvgPool1d(30)
        self.fc1 = nn.Linear(512, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x =  F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = self.avgpool(x)
        x = x.permute(0,2,1)
        x = self.fc1(x)
        
        return F.log_softmax(x, dim=2)
    
    
    

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
audio_net = AudioNet()
audio_net.to(device)

In [ ]:
## model hyper parameters

optimizer = optim.Adam(audio_net.parameters(), lr = 1e-5)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(10):
    audio_net.to(device)
    audio_net.train()
    for out in enumerate(train_loader):
        print(len(out[1]))
        input, label = input.to(device), label.to(device)
        optimizer.zero_grad()
        output = audio_net(input)
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()
        

In [ ]:
import librosa

In [ ]:
sample_data, sr = librosa.load("/home/ifire/learnpytorh/audio/ESC-50/audio/train/1-100032-A-0.wav",sr=None)
spectrogram = librosa.feature.melspectrogram(y=sample_data, sr=sr)

In [ ]:
librosa.display.specshow(spectrogram, sr=sr, x_axis='time', y_axis='mel')

In [ ]:
lof_spectogram = librosa.power_to_db(spectrogram, ref=np.max)

In [ ]:
librosa.display.specshow(lof_spectogram, sr=sr, x_axis='time', y_axis='mel')

because these spectograms are image data then it contains the structure of the audio data
therefore we can use the spectgrams along with CCN2d to capture the sturcture and classify them

In [ ]:
## new data set with spectograms
class ESC50Spectograms(Dataset):
    def __init__(self, path) -> None:
        files = Path(path).glob('*.wav')
        self.items = [(f, int(f.name.split("-")[-1].replace(".wav", ""))) for f in files]
        self.length = len(self.items)
        self.tranforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
        
    def __getitem__(self, index) -> torch.Tensor:
        filename, label = self.items[index]
        audio_tensor, sample_rate = librosa.load(filename, sr=None)
        spectrogram = librosa.feature.melspectrogram(y=audio_tensor, sr= sample_rate)
        lof_spectogram = librosa.power_to_db(spectrogram, ref=np.max)
        librosa.display.specshow(lof_spectogram, sr=sample_rate, x_axis='time', y_axis='mel')
        
        plt.gcf().canvas.draw()
        audio_data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        audio_data = audio_data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        return (self.transforms(audio_data), label)
    
    def __len__(self):
        return self.length
                

In [ ]:
oldESC50  = ESC50(PATH_TO_ESC50/ 'train')
start_time = time.process_time()
oldESC50.__getitem__(33)
end_time = time.process_time()
old_time = end_time - start_time

In [ ]:
newESC50 = ESC50Spectograms(PATH_TO_ESC50/ 'train')
start_time = time.process_time()
newESC50.__getitem__(33)
end_time = time.process_time()
new_time = end_time - start_time

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
writer = SummaryWriter()
writer.add_scalar('example', 3)

In [ ]:
## testing with random values
for i in range(100):
    writer.add_scalar('Loss', np.random.random(), i)
    writer.add_scalar('Accuracy', np.random.random(), i)

In [ ]:
## model representation 
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms, models

In [6]:
writer = SummaryWriter()
model = models.resnet18(False)
writer.add_graph(model, torch.rand(1,3,224,224))


NameError: name 'SummaryWriter' is not defined

In [ ]:
def train(model,
          optimizer,
          loss_fn,
          train_loader,
          test_loader,
          epochs=50):
    model = model.train()
    iteration = 0
    
    for epoch in range(epochs):
        model.to(device)
        model.train()
        for batch in train_loader:
            input, label = batch
            optimizer.zero_grad()
            input, label = input.to(device), label.to(device)
            output = model(input)
            loss = loss_fn(output, label)
            
            writer.add_scalar('loss', loss, epoch)
            loss.backward()
            optimizer.step()
            
        model.eval()
        num_corrects = 0
        num_example = 0
        for batch in test_loader:
            input, label = batch
            input, label = input.to(device), label.to(device)
            output = model(input)
            correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], label).view(-1)
            
            num_corrects += torch.sum(correct).item()
            num_example += correct.shape[0]
            
            print('[INFO] Epoch - {epoch} : accuracy - {num_corrects/num_example}')
        iteration +=1

In [ ]:
def print_hook(self, module, input):
    print(f"Shape of the input is - {input.shape} ")

In [4]:
model = models.resnet18()
hook_ref = model.fc.register_forward_hook(print_hook)
model(torch.rand([1,3,224,224]))
hook_ref.remove()
model(torch.rand([1,3,224,224]))

NameError: name 'models' is not defined

In [ ]:
def sen_to_tensorboard(i, module, input, output):
    writer.add_scalar(f"{i}-mean ", output.data.mean())
    writer.add_scalar(f"{i}-stddev ", output.data.std())
    

In [ ]:
model.children

In [ ]:
for i, m in enumerate(model.children()):
    print(m)

In [ ]:
from functools import partial

for i, m in enumerate(model.children()):
    m.register_forward_hook(partial(sen_to_tensorboard, i))

In [ ]:
train_data = datasets.MNIST(root = ".", train=True, transform=transforms.ToTensor(), download=True)
test_data = datasets.MNIST(root = ".", train=False, transform=transforms.ToTensor(), download=True)

In [ ]:
import torch.utils
from  torch.utils.data import DataLoader


train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=5, drop_last=True, pin_memory=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False, num_workers=5, drop_last=True, pin_memory=True)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4)

In [ ]:
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)

    
input_shape = model.fc.in_features
hidden_units = 128
model.fc = nn.Sequential(
    nn.Linear(input_shape, hidden_units),
    nn.BatchNorm1d(hidden_units),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(hidden_units, 10),
)


In [ ]:
train(model=model, 
      optimizer=optimizer,
      loss_fn=loss_fn,
      train_loader=train_dataloader, 
      test_loader=test_dataloader,
      epochs=10)

In [23]:
class Saveactivations():
    acvtivations = None
    def __init__(self, m):
        self.hook = m.register_forward_hook(self.hook_fn)
    
    def hook_fn(self, module, input, output):
        self.features = output.data
    
    def remove(self):
        self.hook.remove() 

In [14]:
import torch
from torchvision import transforms
from torch.nn import functional as F
from torchvision import models

In [8]:
from PIL import Image

In [15]:
cat = Image.open('pexels-photo-1170986.png')
##then we normalize
normalize = transforms.Normalize(mean  = [0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])

In [16]:
model = models.resnet18()

In [17]:
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)

    
input_shape = model.fc.in_features
hidden_units = 128
model.fc = nn.Sequential(
    nn.Linear(input_shape, hidden_units),
    nn.BatchNorm1d(hidden_units),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(hidden_units, 10),
)


In [20]:
preprocess = transforms.Compose([transforms.Resize((224,224)),
                                 transforms.ToTensor(),
                                 normalize])

In [21]:
display_transforms = transforms.Compose([transforms.Resize((224, 224))])
casper_tensor = preprocess(cat)

In [24]:
model = models.resnet18(pretrained=True)
model.eval()
casper_activations = Saveactivations(model.layer4)
prediction = model(casper_tensor.unsqueeze(0))
pred_probabilities  = F.softmax(prediction).data.squeeze()
casper_activations.remove()
torch.topk(pred_probabilities,1)

/tmp/ipykernel_1716464/1064327656.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_probabilities  = F.softmax(prediction).data.squeeze()


torch.return_types.topk(
values=tensor([0.2894]),
indices=tensor([285]))

In [28]:
print(casper_activations.acvtivations)

None


In [30]:
class SaveActivations():
    activations=None
    def __init__(self, m):
        self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.features = output.data
    def remove(self):
        self.hook.remove()

In [32]:
import torch
from torchvision import models, transforms

In [33]:
casper = Image.open("pexels-photo-1170986.png")

In [34]:
normalize = transforms.Normalize(
 mean=[0.485, 0.456, 0.406],
 std=[0.229, 0.224, 0.225]
)

In [35]:
preprocess = transforms.Compose([
 transforms.Resize((224,224)),
 transforms.ToTensor(),
 normalize
])

In [36]:
display_transform = transforms.Compose([
 transforms.Resize((224,224))])
casper_tensor = preprocess(casper)

In [38]:
model = models.resnet18(pretrained=True)
model.eval()
casper_activations = SaveActivations(model.layer4)
prediction = model(casper_tensor.unsqueeze(0))
pred_probabilities = F.softmax(prediction).data.squeeze()
casper_activations.remove()
torch.topk(pred_probabilities,1)

/home/ifire/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ifire/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_1716464/1944733709.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_probabilities = F.softmax(prediction).data.squeeze()


torch.return_types.topk(
values=tensor([0.2744]),
indices=tensor([281]))

In [41]:
fts = sf[0].features[idx]
    prob = np.exp(to_np(log_prob))
    preds = np.argmax(prob[idx])
    fts_np = to_np(fts)
    f2=np.dot(np.rollaxis(fts_np,0,3), prob[idx])
    f2-=f2.min()
    f2/=f2.max()
    f2

IndentationError: unexpected indent (378078884.py, line 2)